# Optional Add-on Module: Parallelization, Vectorization & Lazy Evaluation in Pandas

In this section, we dive into how **Pandas operations can be accelerated** through:
- **Vectorization** using NumPy operations
- **Parallelization** with libraries like `swifter` and `modin`
- **Lazy evaluation** using `dask.dataframe` for large datasets

You'll learn how to handle large-scale data pipelines efficiently, while understanding the tradeoffs between memory, performance, and execution control.

## 1. Vectorization in Pandas

Vectorization allows operations to be applied over arrays without explicit loops. This uses optimized C-level implementations inside NumPy and Pandas.

In [ ]:
import pandas as pd
import numpy as np
import time

# Create synthetic dataset
df = pd.DataFrame({
    'A': np.random.randint(1, 100, 1_000_000),
    'B': np.random.randint(1, 100, 1_000_000)
})

# Non-vectorized operation (slow)
start = time.time()
df['C_loop'] = [a * b for a, b in zip(df['A'], df['B'])]
loop_time = time.time() - start

# Vectorized operation (fast)
start = time.time()
df['C_vec'] = df['A'] * df['B']
vec_time = time.time() - start

print(f"Loop time: {loop_time:.3f}s | Vectorized time: {vec_time:.3f}s")

✅ **Observation:** Vectorized code is **10–100× faster** and uses internal C loops.

### Under the Hood: Why Vectorization Wins
- Operates directly on **NumPy ndarray buffers**.
- Uses **SIMD (Single Instruction, Multiple Data)** CPU instructions.
- Avoids Python's GIL by executing C extensions natively.

Vectorization is the foundation of high-performance data processing in Python.

## 2. Parallelization using Swifter & Modin

When your functions are not easily vectorizable (like complex Python logic), libraries such as **Swifter** and **Modin** distribute Pandas operations across multiple CPU cores.

In [ ]:
# Example: Accelerating apply() with Swifter
!pip install -q swifter
import swifter

# Apply custom function
def complex_operation(x):
    return np.sqrt(x**2 + np.log1p(x))

# Using swifter to parallelize apply()
df['D'] = df['A'].swifter.apply(complex_operation)
df.head()

### Real-World Problem 1: Parallel Customer Scoring

**Scenario:** You have millions of customer transactions and need to compute a risk score using a non-vectorizable function.

```python
def risk_score(amount, frequency):
    return np.log1p(amount) * np.sqrt(frequency)

transactions['risk'] = transactions.swifter.apply(lambda row: risk_score(row['amount'], row['frequency']), axis=1)
```

⏱ Swifter automatically parallelizes across available cores for faster results.

## 3. Lazy Evaluation with Dask DataFrame

Lazy evaluation means operations are **queued** until you explicitly trigger computation. This helps process datasets larger than memory by chunking them into partitions.

In [ ]:
!pip install -q dask
import dask.dataframe as dd

# Create a Dask DataFrame
dask_df = dd.from_pandas(df, npartitions=8)

# Lazy operations (no immediate execution)
result = (dask_df['A'] + dask_df['B']).mean()

# Trigger computation
print(result.compute())

### Real-World Problem 2: Processing a 5GB CSV File

**Scenario:** You have a 5GB CSV file that doesn't fit in memory. Using Dask allows chunked reading and lazy evaluation.

```python
import dask.dataframe as dd
big_df = dd.read_csv('sales_data_2024.csv')
filtered = big_df[big_df['revenue'] > 10000]
avg = filtered['revenue'].mean()
print(avg.compute())  # Executes only when needed
```

✅ Efficient for large-scale ETL and analytics pipelines.

## 4. Profiling and Benchmarking

Let's measure and compare performance using built-in tools like `%timeit` and `memory_usage()`.

In [ ]:
# Compare vectorized vs. swifter performance
%timeit df['A'] * df['B']
%timeit df['A'].swifter.apply(complex_operation)

# Memory usage estimation
print(df.memory_usage(deep=True).sum() / (1024**2), 'MB')

## Under the Hood
- **Dask:** Divides DataFrame into smaller Pandas partitions and uses a task graph scheduler.
- **Swifter:** Uses `pandarallel` or `dask` under the hood depending on context.
- **NumExpr:** Can further optimize numerical expressions with JIT compilation.
- **Modin:** Distributes Pandas operations seamlessly across CPU cores or Ray/Dask clusters.

## Best Practices & Pitfalls

✅ **Do:**
- Prefer vectorized operations whenever possible.
- Profile both time and memory before scaling.
- Use Dask for large data that doesn't fit in memory.
- Use Swifter/Modin when apply-based logic is unavoidable.

⚠️ **Avoid:**
- Mixing Pandas and Dask APIs in the same pipeline without care.
- Over-partitioning Dask DataFrames (too many small tasks can degrade performance).
- Expecting `apply()` to parallelize automatically—it won’t without helper libraries.

## Challenge Exercise

**Task:** Load a 2M-row CSV file of user activity logs and:
1. Compute average session duration per user using **vectorized operations**.
2. Apply a custom scoring function using **Swifter**.
3. Re-implement the pipeline using **Dask** to process lazily and compare execution time.

_Hint:_ Use `dd.read_csv()` and `.compute()` for final evaluation.

# --- End of Add-on Module Section 2 ---